# 2D advection PINN 

In [1]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import math
import os
import imageio

# Define the Neural architecture

In [2]:
class FCN(nn.Module):
    "Defines a connected network"
    
    def __init__(self, N_INPUT, N_OUTPUT, N_HIDDEN, N_LAYERS, dropout_rate=1e-3):
        super().__init__()
        activation = nn.Tanh
        self.fcs = nn.Sequential(
            nn.Linear(N_INPUT, N_HIDDEN),
            activation(),
            nn.Dropout(dropout_rate)
        )
        self.fch = nn.Sequential(
            *[
                nn.Sequential(
                    nn.Linear(N_HIDDEN, N_HIDDEN),
                    activation(),
                    nn.Dropout(dropout_rate)
                ) for _ in range(N_LAYERS-1)
            ]
        )
        self.fce = nn.Linear(N_HIDDEN, N_OUTPUT)
        
    def forward(self, x):
        x = self.fcs(x)
        x = self.fch(x)
        x = self.fce(x)
        
        return x

# Advection equation 

$ -\left( v(x) \frac{\partial u}{\partial x} + v(y) \frac{\partial u}{\partial y} \right)$


# u0 = gaussian pulse | u2d has added y component


In [3]:
mu = 0.1 
def u0(x):
    return np.exp(-100*(x-0.2)**2)
def u_true(x,t):
    return  u0(x - mu * t)
# 2D advection equation with gaussian initial condition
def u_2d_true(x, y, t):
        return u0(x - mu * t) * u0(y - mu * t)

# Check input in x-y plot

In [4]:
xx = np.linspace(0, 1, 100).reshape(-1,1)
print(xx.shape)
plt.figure(figsize=(5,2))
for tt in range(4):
    plt.plot(xx, u_true(xx, 0), '-', color=cm.rainbow(tt/4), label="t="+str(tt))
plt.legend()            
plt.show()

# Check input in contour plot

In [5]:
# Define the spatial and temporal coordinates
x = np.linspace(-1, 1, 40)
y = np.linspace(-1, 1, 40)

# Create a meshgrid of the spatial coordinates
X, Y = np.meshgrid(x, y)
print('x shape is', X.shape)
# Define the time values
t_values = np.linspace(0, 4, 5)
print('t values shape is', t_values.shape)
# Plot the solution for each time value
for t in t_values:
    # Compute the solution at the given time
    U = u_2d_true(X, Y, t)
    print('U shape is',U.shape)
    # Plot the solution
    plt.figure(figsize=(6, 5))
    plt.contourf(X, Y, U, cmap='viridis')
    plt.colorbar(label='u')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('Solution to the 2D advection equation at t = {}'.format(t))
    plt.show()


# Define training data [xyt_data] and loss [u_data]

In [6]:
'''
# Define the spatial and temporal coordinates
nx,ny as x and y resolution
nt as time resolution
xyt_data is inital np array of x,y,t columns
X_t, Y_t is meshgrid of x,y
u_data is the u_2d_true function output
Pyt_data is the meshgrid of X_t,Y_t,t

'''
nx,ny, nt = 10, 10,3

xyt_data = np.hstack((
    np.repeat([np.linspace(0, 1, nx) ], nt, axis=0).reshape(-1,1),
    np.repeat([np.linspace(0, 1, ny) ], nt, axis=0).reshape(-1,1),
    np.repeat([np.linspace(0,nt-1,nt)], nx, axis=0).T.reshape(-1,1)))


X_t,Y_t = np.meshgrid(xyt_data[:,0], xyt_data[:,1])

Pyt_data = np.vstack((X_t.flatten(), Y_t.flatten(),np.repeat( [xyt_data[:,2]],nx*nt,axis=0).flatten())).T

u_data = u_2d_true(X_t,Y_t, xyt_data[:,2])

# Convert to torch tensors  and float
Pyt_data = torch.tensor(Pyt_data).float()
xyt_data = torch.tensor(xyt_data).float()
u_data = torch.tensor(u_data).float().reshape(-1,1)

print(u_data.shape)
print(Pyt_data.shape, u_data.shape)

N_OUTPUT = nx*nt


In [7]:
xr = np.linspace(0,1,nx)
yr = np.linspace(0,1,ny)
tr = np.linspace(0,nt-1,nt).T
# print(f'xr: {xr} \n yr: {yr} \n tr: {tr}')
# print(f'xrshape: {xr.shape} \n yrshape: {yr.shape} \n trshape: {tr.shape}')
xrmesh , yrmesh, trmesh = np.meshgrid(xr,yr,tr)
# print(f'xrmesh: {xrmesh} \n yrmesh: {yrmesh} \n trmesh: {trmesh}')
# print(f'xrmeshshape: {xrmesh.shape} \n yrmeshshape: {yrmesh.shape} \n trmeshshape: {trmesh.shape}')

ur = u_2d_true(xrmesh,yrmesh,trmesh)
# print(ur.shape)
# print(ur[0,:,:].shape)
# print(xrmesh[0,:,:].shape)
#plot ur
plt.tricontourf(xrmesh.flatten(), yrmesh.flatten(), ur.flatten())
plt.xlabel('X')
plt.ylabel('Y')
plt.colorbar()
plt.show()

# Stack the 3 2D arrays along a new third dimension, then reshape into a 2D array
rin_data = np.stack((xrmesh, yrmesh, trmesh), axis=-1).reshape(-1, 3)
rin_data = torch.tensor(rin_data).float()
rout_data = torch.tensor(ur).float().reshape(-1,1)

print(f'rin_data shape: {rin_data.shape} \n rout_data shape: {rout_data.shape}')


# plot the tensor form of input data and loss function 

In [8]:
'''
plot the 3D plot,Contour plot of the xyt_data and u_data

'''
fig = plt.figure(figsize=(12, 6))
ax = fig.add_subplot(111, projection='3d')


print(u_2d_true(X, Y, 0).shape)
print(X.shape, Y.shape)

# xt,yt is the meshgrid of x and y from xyt_data
xt ,yt = np.meshgrid(xyt_data[:,0].detach().numpy(), xyt_data[:,1].detach().numpy())

print(u_data.shape)

# Reshape the u_data to 2D with same shape as xt,yt
uuu = u_data.detach().numpy().reshape(N_OUTPUT,N_OUTPUT)

print(uuu.shape)

# Plot for t=1
ax.plot_surface(xt, yt,uuu , cmap='viridis', alpha=0.5)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('u')
ax.set_title('3D Plot at t=1')

plt.tight_layout()
plt.show()
plt.contourf(xt,yt, uuu, cmap='viridis')

# Define parameters of Neural Network

In [9]:
torch.manual_seed(123)
# N_INPUT, N_OUTPUT, N_HIDDEN, N_LAYERS
model = FCN(3,1,32,3)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-1)
for i in range(5000):
    optimizer.zero_grad()
    uh = model(rin_data)  # Transpose the input matrix
    
    loss = torch.mean((uh-rout_data)**2)
    loss.backward()
    optimizer.step()

# Test model 

In [10]:
xr = np.linspace(0,1,nx)
yr = np.linspace(0,1,ny)
tr = np.linspace(0,nt-1,nt)
# print(f'xr: {xr} \n yr: {yr} \n tr: {tr}')
# print(f'xrshape: {xr.shape} \n yrshape: {yr.shape} \n trshape: {tr.shape}')
xrmesh , yrmesh, trmesh = np.meshgrid(xr,yr,tr)
# print(f'xrmesh: {xrmesh} \n yrmesh: {yrmesh} \n trmesh: {trmesh}')
# print(f'xrmeshshape: {xrmesh.shape} \n yrmeshshape: {yrmesh.shape} \n trmeshshape: {trmesh.shape}')
torch.manual_seed(123)
urnn = model(rin_data).detach().numpy()
# print(ur.shape)
# print(ur[0,:,:].shape)
# print(xrmesh[0,:,:].shape)
#plot ur
plt.tricontourf(xrmesh.flatten(), yrmesh.flatten(), urnn.flatten(),cmap=cm.jet)
plt.xlabel('X')
plt.ylabel('Y')
plt.colorbar()
plt.show()

plt.tricontourf(xrmesh.flatten(), yrmesh.flatten(), ur.flatten(),cmap=cm.jet)
plt.xlabel('X')
plt.ylabel('Y')
plt.colorbar()
plt.show()


In [11]:
'''
# Define the spatial and temporal coordinates
nx,ny as x and y resolution
nt as time resolution
xyt_test is inital np array of x,y,t columns
X_t, Y_t is meshgrid of x,y
u_test is the u_2d_true function output
Pyt_test is the meshgrid of X_t,Y_t,t

'''
# Define the spatial and temporal coordinates
xyt_test = np.hstack((
    np.repeat([np.linspace(-1, 1, nx) ], nt, axis=0).reshape(-1,1),
    np.repeat([np.linspace(-1, 1, ny) ], nt, axis=0).reshape(-1,1),
    np.repeat([np.linspace(0,nt-1,nt)], nx, axis=0).T.reshape(-1,1)))

# Create a meshgrid of the spatial coordinates
X_t,Y_t = np.meshgrid(xyt_data[:,0], xyt_data[:,1])

# py_test is the meshgrid of X_t,Y_t,t
Pyt_test = np.hstack((X_t.reshape(-1,1), Y_t.reshape(-1,1),np.repeat( [xyt_test[:,2]],nx*nt,axis=0).reshape(-1,1))).T

# Convert to torch tensors  and float
xyt_test = torch.tensor(xyt_data).float().reshape(-1,3)
Pyt_test = torch.tensor(Pyt_data).float().reshape(-1,3)

'''
Generate and save the plots for tt = 0-11
'''
cwd = os.getcwd()
true_plot_dir = os.path.join(cwd, '2d-advection', 'true_plot')
nn_plot_dir = os.path.join(cwd, '2d-advection', 'nn_plot')

# Create the directories if they don't exist
os.makedirs(nn_plot_dir, exist_ok=True)
os.makedirs(true_plot_dir, exist_ok=True)

for tt in range(10):
    # Compute the solution at the given time
    xyt_test[:, 2] = tt
    col = cm.rainbow(tt / 5)
    u_test = u_2d_true(X_t, Y_t, xyt_test[:, 2].numpy())

    m = model(Pyt_test).detach().numpy().reshape(N_OUTPUT, N_OUTPUT)
    print(m.shape)
    
    # Plot and save the nn plot
    plt.contourf(X_t, Y_t,m , cmap='viridis', alpha=0.5)
    plt.colorbar(label='u')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('NN Plot at t = {}'.format(tt))
    plt.savefig(nn_plot_dir + '/nn_plot_{}.png'.format(tt))
    plt.close()
    
    # Plot and save the true plot
    plt.contourf(X_t, Y_t, u_test, cmap='viridis', alpha=0.5)
    plt.colorbar(label='u')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('True Plot at t = {}'.format(tt))
    plt.savefig(true_plot_dir + '/true_plot_{}.png'.format(tt))
    plt.close()

# Save GIF

In [12]:
'''
Saves the plots as a GIF
'''
for _ in ['nn_plot', 'true_plot']:
    # Set the directories for the images
    plot_dir = os.path.join(cwd, '2d-advection', _)

    # Get the list of image files
    plot_files = sorted([os.path.join(plot_dir, file) for file in os.listdir(plot_dir)])

    # Create a list to store the images
    images = []

    # Read and append the images to the list
    for file in plot_files:
        images.append(imageio.imread(file))

    # Set the output file path
    output_file = os.path.join(cwd, "2d{}.gif".format(_))

    # Save the images as a GIF
    imageio.mimsave(output_file, images, duration=1, loop=10, fps=1, subrectangles=True, palettesize=256)



# Define physics sampling points

In [13]:
# randomly sample points in x,y-t space, where x & y in [0,1] and t in [0,11]
np.random.seed(123)
xyt_physics = np.random.uniform(size=(nx*ny*nt,nt))
print(xyt_physics.shape)
xyt_physics[:,2] *= 3
xyt_physics = torch.tensor(xyt_physics).float().requires_grad_(True)

# Define neural network parameters and train 

In [14]:
'''
# Define the spatial and temporal coordinates
nx,ny as x and y resolution
nt as time resolution
X_t, Y_t is meshgrid of x,y
u_test is the u_2d_true function output
Pyt_test is the meshgrid of X_t,Y_t,t
xyt_physics is the random sample of x,y,t 
'''

torch.manual_seed(123)
# N_INPUT, N_OUTPUT, N_HIDDEN, N_LAYERS
pinn = FCN(3,1,16,3)

optimizer = torch.optim.Adam(pinn.parameters(), lr=0.01)
for i in range(10000):
    optimizer.zero_grad()
    
    # compute the "data loss"
    uh = pinn(rin_data)
    
    loss1 = torch.mean((uh-rout_data)**2) # use mean squared error
    
    # compute the "physics loss"
    uhp = pinn(xyt_physics)
    grad = torch.autograd.grad(outputs=uhp, inputs=xyt_physics, 
                               grad_outputs=torch.ones_like(uhp), create_graph=True)[0]
    
    dudx = grad[:, 0]
    dudy = grad[:, 1]
    dudt = grad[:, 2]
    physics = dudt + mu * dudx + mu * dudy # this term is zero if the model output satisfies the advection equation
    loss2 = torch.mean((physics)**2)
    # multiply by a large number to make it comparable to loss1
    loss2 *= 1e6
    # backpropagate combined loss
    loss =  loss1 + loss2 # add two loss terms together
    loss.backward()
    optimizer.step()

    if (i+1) % 1000 == 0:
            print(f'Epoch: {i+1}/{10000}, Loss: {loss.item()}')
            print(f' NN loss : {loss1.item()}, Physics loss : {loss2.item()}')
            

# Test Pinn

In [15]:
xr = np.linspace(0,1.2,nx)
yr = np.linspace(0,1.2,ny)
tr = np.linspace(0,nt-1,nt)
# print(f'xr: {xr} \n yr: {yr} \n tr: {tr}')
# print(f'xrshape: {xr.shape} \n yrshape: {yr.shape} \n trshape: {tr.shape}')
xrmesh , yrmesh, trmesh = np.meshgrid(xr,yr,tr)
# print(f'xrmesh: {xrmesh} \n yrmesh: {yrmesh} \n trmesh: {trmesh}')
# print(f'xrmeshshape: {xrmesh.shape} \n yrmeshshape: {yrmesh.shape} \n trmeshshape: {trmesh.shape}')
rin_data = np.stack((xrmesh, yrmesh, trmesh), axis=-1).reshape(-1, 3)
rin_data = torch.tensor(rin_data).float()
print(trmesh.shape)

ur = u_2d_true(xrmesh,yrmesh,trmesh)
torch.manual_seed(123)
for i in range(5000):
    urnn = pinn(rin_data).detach().numpy().reshape(nx,ny,nt)
    error = np.mean(np.abs(urnn - ur)**2)
    
    
    if error < 0.008:
        for j in range(nt):
            plt.contourf(xrmesh[:,:,j], yrmesh[:,:,j], urnn[:,:,j],cmap=cm.jet)
            plt.xlabel('X')
            plt.ylabel('Y')
            plt.title(f'NN Solution at Epoch {i} and t={j}')
            plt.colorbar()
            plt.show()

            ur = u_2d_true(xrmesh,yrmesh,trmesh)
            plt.contourf(xrmesh[:,:,j], yrmesh[:,:,j], ur[:,:,j],cmap=cm.jet)
            plt.xlabel('X')
            plt.ylabel('Y')
            plt.title(f'True Solution at t={j}')
            plt.colorbar()
            plt.show()
            


In [16]:
'''
# Define the spatial and temporal coordinates
nx,ny as x and y resolution
nt as time resolution
xyt_test is inital np array of x,y,t columns
X_t, Y_t is meshgrid of x,y
u_test is the u_2d_true function output
Pyt_test is the meshgrid of X_t,Y_t,t

'''
# Define the spatial and temporal coordinates
xyt_test = np.hstack((
    np.repeat([np.linspace(-1, 1.4, nx) ], nt, axis=0).reshape(-1,1),
    np.repeat([np.linspace(-1, 1.4, ny) ], nt, axis=0).reshape(-1,1),
    np.repeat([0.0],N_OUTPUT).reshape(-1,1)))

# Create a meshgrid of the spatial coordinates
X_t,Y_t = np.meshgrid(xyt_test[:,0], xyt_test[:,1])

# py_test is the meshgrid of X_t,Y_t,t
Pyt_test = np.hstack((X_t.reshape(-1,1), Y_t.reshape(-1,1),np.repeat( [xyt_test[:,2]],nx*nt,axis=0).reshape(-1,1))).T

# Convert to torch tensors  and float
xyt_test = torch.tensor(xyt_data).float().reshape(-1,3)
Pyt_test = torch.tensor(Pyt_data).float().reshape(-1,3)

'''
Generate and save the plots for tt = 0-11
'''
cwd = os.getcwd()
true_plot_dir = os.path.join(cwd, '2d-advection', 'true_plot')
nn_plot_dir = os.path.join(cwd, '2d-advection', 'pinn_plot')

# Create the directories if they don't exist
os.makedirs(nn_plot_dir, exist_ok=True)
os.makedirs(true_plot_dir, exist_ok=True)

for tt in range(10):
    # Compute the solution at the given time
    Pyt_test[:, 2] = tt
    xyt_test[:, 2] = tt
    col = cm.rainbow(tt / 5)
    u_test = u_2d_true(X_t, Y_t, xyt_test[:, 2].numpy())
    m = pinn(Pyt_test).detach().numpy().reshape(N_OUTPUT, N_OUTPUT)

    # Plot and save the nn plot
    plt.contourf(X_t, Y_t, m, cmap='viridis', alpha=0.5)
    plt.colorbar(label='u')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('NN Plot at t = {}'.format(tt))
    plt.savefig(nn_plot_dir + '/pinn_plot_{}.png'.format(tt))
    plt.close()
    


    # Plot and save the true plot
    plt.contourf(X_t, Y_t, u_test, cmap='viridis', alpha=0.5)
    plt.colorbar(label='u')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('True Plot at t = {}'.format(tt))
    plt.savefig(true_plot_dir + '/true_plot_{}.png'.format(tt))
    plt.close()
    print('done',tt)



# Save Pinn

In [17]:
'''
Saves the plots as a GIF
'''
for _ in ['pinn_plot', 'true_plot']:
    # Set the directories for the images
    plot_dir = os.path.join(cwd, '2d-advection', _)

    # Get the list of image files
    plot_files = sorted([os.path.join(plot_dir, file) for file in os.listdir(plot_dir)])

    # Create a list to store the images
    images = []

    # Read and append the images to the list
    for file in plot_files:
        images.append(imageio.imread(file))

    # Set the output file path
    output_file = os.path.join(cwd, "2d{}.gif".format(_))

    # Save the images as a GIF
    imageio.mimsave(output_file, images, duration=1, loop=10, fps=1, subrectangles=True, palettesize=256)
